# Which MobileNet layers take how much time?

We want to choose a MobileNet inverted residual block to benchmark. A reasonable way to do this is to select the one that takes the most FLOPs of the model. 

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), include_top=True)

In [ ]:
# from thesis.benchmark import get_keras_model_flops

In [ ]:
def get_keras_model_flops(keras_model, batch_size=1):
    from tensorflow.python.profiler.model_analyzer import profile
    from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

    # model = tf.keras.applications.ResNet50()
    forward_pass = tf.function(
        keras_model.call,
        input_signature=[
            tf.TensorSpec(shape=[batch_size] + list(keras_model.input.shape)[1:])
        ],
    )

    opts = ProfileOptionBuilder().float_operation()
    # Silence stdout output
    # opts["output"] = "none"

    graph_info = profile(
        forward_pass.get_concrete_function().graph,
        options=opts,
    )

    return graph_info, graph_info.total_float_ops


In [ ]:
model.input.shape

In [ ]:
graph_info, flops = get_keras_model_flops(model)

In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
rows = []

for child in graph_info.children:
    rows.append({
        "name": child.name,
        "float_ops": child.float_ops,
    })

df = pd.DataFrame(rows)

In [ ]:
df["block"] = df["name"].str.extract("block_([0-9]+)_").fillna(0).astype(int)

In [ ]:
df = df.sort_values(["block", "name"])

In [ ]:
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (10,20)

sns.barplot(data=df, y="name", x="float_ops")

MobileNet paper: https://arxiv.org/pdf/1801.04381.pdf

The blocks here correspond to:

| Blocks | Input size        |
| ---    | ---               |
| 1      | $112^2 \times 32$ |
| 2-3    | $112^2 \times 16$ |
| 4-6    | $56^2 \times 24$ |
| 7-10   | $28^2 \times 32$ |
| 11-13  | $14^2 \times 64$ |
| 14-16  | $14^2 \times 96$ |

In [ ]:
groups = [[1], [2,3], [4,5,6], [7,8,9,10], [11,12,13], [14,15,16]]

In [ ]:
df["group"] = 0
for i, group in enumerate(groups):
    df.loc[df["block"].isin(group), "group"] = i + 1

In [ ]:
df.groupby("group").agg({"float_ops": "sum"})

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)

sns.barplot(data=df.groupby("group").agg({"float_ops": "sum"}).reset_index(), y="float_ops", x="group")

Conclusion: group 5 (input size 14x14x96) has the highest flops of the model